In [1]:
import csv 
import pandas as pd
import numpy as np

CSV File Paths (this will need to be changed if the datasets are located in another directory on your machine)

In [2]:
facilities_path = 'facilities.csv'
banks_path = 'banks.csv'
covenants_path = 'covenants.csv'
loans_path = 'loans.csv'

Using Pandas to read CSV files into Dataframes. This cell also takes care of making the columns unique among all dataframes.

In [3]:
DF_FACILITIES = pd.read_csv(facilities_path).rename(columns = {'id':'facility_id', 'amount':'facility_amount', 'interest_rate':'facility_interest_rate'})
DF_BANKS = pd.read_csv(banks_path)
DF_COVENANTS = pd.read_csv(covenants_path).fillna(1)
DF_LOANS = pd.read_csv(loans_path).rename(columns = {'id':'loan_id', 'interest_rate':'loan_interest_rate'})

In [4]:
def get_facility_dictionary(FACILITY_DATA):
    FACILITY_DICT = {}
    for index, facility in FACILITY_DATA.iterrows():
        FACILITY_DICT[int(facility['facility_id'])] = int(facility['facility_amount'])
    return FACILITY_DICT

In [5]:
def create_assignment_table(LOAN_DATA, COVENANT_DATA, FACILITY_DATA):
    OPTIMAL_COVENANT = pd.merge(COVENANT_DATA, FACILITY_DATA, on = 'facility_id').sort_values(by=['facility_amount','facility_interest_rate'])
    DF_ASSIGNMENTS =  pd.DataFrame(columns=['loan_id', 'facility_id']) 
    FACILITY_DICT = get_facility_dictionary(FACILITY_DATA)
    for index, loan in LOAN_DATA.iterrows():
        for index, covenant in OPTIMAL_COVENANT .iterrows():
            difference = FACILITY_DICT.get(covenant['facility_id']) - loan['amount']
            if (loan['default_likelihood'] < covenant['max_default_likelihood']) and (loan['state'] != covenant['banned_state']) and difference >= 0:
                FACILITY_DICT[covenant['facility_id']] = difference
                DF_ASSIGNMENTS = DF_ASSIGNMENTS.append({'loan_id': loan['loan_id'], 'facility_id':covenant['facility_id']}, ignore_index=True)
                break
    return DF_ASSIGNMENTS

In [6]:
def get_expected_yield(facility_id, default_likelihood, loan_interest_rate, amount, facility_interest_rate):
    return (int(facility_id), int((1 - default_likelihood) * (loan_interest_rate * amount) - (default_likelihood * amount) - (facility_interest_rate * amount)))

In [7]:
def clean_yield_data(assigned_csv, LOAN_DATA, FACILITY_DATA):
    ASSIGNED_DATA = pd.read_csv(assigned_csv)
    ASSIGNED_LOANS = pd.merge(ASSIGNED_DATA, LOAN_DATA, on='loan_id')
    ALL_TOGETHER_NOW = pd.merge(FACILITY_DATA, ASSIGNED_LOANS, on = 'facility_id')
    return ALL_TOGETHER_NOW

In [8]:
def create_yeild_table(assigned_csv, LOAN_DATA, FACILITY_DATA):
    ALL_TOGETHER_NOW = clean_yield_data(assigned_csv, LOAN_DATA, FACILITY_DATA)
    ALL_TOGETHER_NOW['yield'] = ALL_TOGETHER_NOW.apply(lambda x: get_expected_yield(x['facility_id'],x['default_likelihood'], 
                                                                                    x['loan_interest_rate'], x['amount'], 
                                                                                    x['facility_interest_rate']), axis=1)
    DF_YIELD = ALL_TOGETHER_NOW['yield'].apply(pd.Series)
    DF_YIELD.columns=['facility_id','expected_yield']
    DF_YIELD = DF_YIELD.groupby(['facility_id']).sum().reindex()
    return DF_YIELD

In [9]:
def create_output(DF_LOANS, DF_COVENANTS, DF_FACILITIES):
    df_test = create_assignment_table(DF_LOANS, DF_COVENANTS, DF_FACILITIES)
    df_test.to_csv(path_or_buf = 'assignments.csv', index=False)
    yield_test = create_yeild_table('assignments.csv', DF_LOANS, DF_FACILITIES)
    yield_test.to_csv(path_or_buf = 'yields.csv')
    return None

In [10]:
create_output(DF_LOANS, DF_COVENANTS, DF_FACILITIES)

### Write Up ###

**1. How long did you spend working on the problem? What did you find to be the most
difficult part?**

I spent roughly four hours working on this project. I think the most difficult part was keeping track of the details in regards to which CSV had the data necessary to drive calculations. For example, I would look for the interest necessary to make a calculation by using an id number from the covenant dataset, then connecting that information to the facility dataset. It was difficult to keep straight. However, once I understood the significance of each data set, and how they were all related, it made the work a lot easier. I wouldn't consider it 'difficult', but renaming the datframes in order to be able to accessing the information in a clear way later down the line was something I always had to keep in mind. I've dealt with similar cases of very generalized column names, and having to make them more unique, I just wish I had it in the front of my mind when embarking on the task. The other greatest challenged I faced, was learing new domain knowledge I have not had previous experience with. Without having worked with data like this before, I did not understand what should be maximized/minimized at first. As it stands, the program is not fully correct, but I am eager to discuss the task, and would love the opportunity to explain my work further. 

**2. How would you modify your data model or code to account for an eventual introduction
of new, as-of-yet unknown types of covenants, beyond just maximum default likelihood
and state restrictions?**

In order to make this program more modular, and able to handle more configurations of data, I would have to pull even more of the cleaning processes outside the functions that perform the calculations. It's okay as of right now, but I would want it to be clearer. I think it would also be valuable to extract the sort method (currently I'm using the pandas built in 'sort_values' method), and allow the user to input the conditions they would like to sort as a list. This structure would allow someone to give the 1st, 2nd ... Nth most important attribute on which to sort. Since the length would only be excluded to the features in the covenant, it would give maximum flexibility. 

**3. How would you architect your solution as a production service wherein new facilities can
be introduced at arbitrary points in time. Assume these facilities become available by the
finance team emailing your team and describing the addition with a new set of CSVs.**

My solution could easily handle more facilities, since the facilities data ultimately gets merged with the covenant data for analysis. This allows the facility information to be easily acccessed when assigning loans to covenants. As long as there is a covenant in existance, for any new facility, it will be used in analysis when assigning loans. 

**4. Your solution most likely simulates the streaming process by directly calling a method in
your code to process the loans inside of a for loop. What would a REST API look like for
this same service? Stakeholders using the API will need, at a minimum, to be able to
request a loan be assigned to a facility, and read the funding status of a loan, as well as
query the capacities remaining in facilities.**

A REST API for this service would be able to call this program, with the only parameter given being the loan. It would return the funding status, with the ability to show the remaining balance of the facility. All other information regarding the covenants, facilities and banks would be managed in-house. There would be no explicit manipulation, or input, of any information other than the loan being requested. The greatest difference between the program with how it stands, and what a REST API implementation would look like, is the amount of manual input the program currently requires. 

**5. How might you improve your assignment algorithm if you were permitted to assign loans
in batch rather than streaming? We are not looking for code here, but pseudo code or
description of a revised algorithm appreciated.**

If the loans were given in batches, I could implement smarter comparisons between members of the subsets, as well as change which loan is assigned first. Depending on the size of the batches, it may even be possible to calculate the expected yield, then maximize it based on the permutations of the order of the subset. However that would only really be feasible if the batches were less than six loans long. Past that, the runtime complexity may be too cumbersome for the advantage to be worth it. So considering the batches are of a hefty length, the first thing I would do is order the loans based on their interest rate, giving the loans with the hightest interest rate the first opportunities to match with the facilities of the lowest interest rate. I believe this would greatly improve the expected yield. However, it is possible that matching the lowest facility interest rate, with the lowest loan interest rate would actually return better results. It really depends on the data, and would require more exploration.  

**6. Discuss your solution’s runtime complexity.**

I'll be honest, there are quite a few for loops, but with good reasoning behind them. For example, when I deciding how to access the current amount left in a facility, I realized the list of facilites was extremely short, but the number of times I would need to access that information would be as long as the number of loans I had. This led me to build a dictionary to house the amount of funding left per facility, knowing it would cause an O(n) function runtime, but taking into consideration n was measurable, and only 15. This seemed to be a fair compromise, because the dictionary proved to be extremly helpful.The main inefficiency in this program is within the function that assigns the loans to facilities based on the covenants. There is a neseted for loop, that iterates through two datframes which makes it extremely clunky and slow, adding O(n^2) to the runtime. This is especially concerning since the two datasets that are being iteratted through could potentially be endless. If I had more time on this project, I would definitely look into using faster built-in functions inside of Pandas, like the lamda function I used within the create yield table function. 